In [1291]:
%pip install python-metar

Note: you may need to restart the kernel to use updated packages.


In [1292]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from metar import Metar
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [1293]:
bimtra_df = pd.read_csv('data/bimtra_df.csv')
esperas_df = pd.read_csv('data/esperas_df.csv')
metaf_df = pd.read_csv('data/metaf_df.csv')
metar_df = pd.read_csv('data/metar_df.csv')
tc_prev_df = pd.read_csv('data/tc-prev_df.csv')
tc_real_df = pd.read_csv('data/tc-real_df.csv')
test_data = pd.read_csv('data/idsc_test.csv')

In [1294]:
bimtra_df.dt_arr = bimtra_df.dt_arr.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
bimtra_df.dt_dep = bimtra_df.dt_dep.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
esperas_df.hora = esperas_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
metaf_df.hora = metaf_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
metar_df.hora = metar_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
tc_prev_df.hora = tc_prev_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
tc_real_df.hora = tc_real_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))

In [1295]:
test_data.head()

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,path,hora_esperas,esperas,aero_esperas,...,metaf,aero_metaf,hora_metar,metar,aero_metar,hora_tcp,troca,aero_tcp,hora_tcr,aero_tcr
0,7945735584a3297121c4f5ae0de8ecd1,SBKP,SBSV,2023-05-30 11:36:03.000,2023-05-30 11:00:00.000,MULTIPOINT ((-0.8807200448127578 -0.5148453155...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-30 10:00:00.000,0,SBSV,...,NaN,NaN,2023-05-30 11:00:00.000,METAR SBSV 301100Z 12009KT 9999 FEW023 28/21 Q...,SBSV,2023-05-30 12:00:00.000,0,SBSV,NaN,NaN
1,85cb958e8b9d583575cd9cb93e389226,SBFL,SBRJ,2023-05-19 08:26:06.000,2023-05-19 08:00:00.000,MULTIPOINT ((-0.8826266039383116 -0.4926305156...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-19 07:00:00.000,0,SBRJ,...,METAF SBRJ 190900Z 33004KT 9999 BKN033 ...,SBRJ,2023-05-19 08:00:00.000,METAR SBRJ 190800Z 02004KT 9999 FEW010 SCT040 ...,SBRJ,2023-05-19 09:00:00.000,0,SBRJ,NaN,NaN
2,2d4e8cd3947525a12ff16af520beefb9,SBCF,SBGR,2023-05-22 11:52:02.000,2023-05-22 11:00:00.000,MULTIPOINT ((-0.8972512464147154 -0.5215196550...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-22 10:00:00.000,0,SBGR,...,METAF SBGR 221200Z 04003KT CAVOK 19/09 Q1021=\n,SBGR,2023-05-22 11:00:00.000,METAR SBGR 221100Z 07006KT CAVOK 13/12 Q1022=,SBGR,2023-05-22 12:00:00.000,0,SBGR,NaN,NaN
3,24cf317a1494a016a018d9f5cd183639,SBFL,SBSP,2023-05-29 16:07:07.000,2023-05-29 16:00:00.000,MULTIPOINT ((-0.892972783728755 -0.51704889771...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-29 15:00:00.000,2,SBSP,...,METAF SBSP 291700Z 15010KT 9999 OVC033 ...,SBSP,2023-05-29 16:00:00.000,METAR SBSP 291600Z 12010KT 090V160 9999 SCT015...,SBSP,2023-05-29 17:00:00.000,0,SBSP,NaN,NaN
4,370babaf32dc82a6a64bf5b71a3434ed,SBCF,SBKP,2023-05-25 22:16:04.000,2023-05-25 22:00:00.000,MULTIPOINT ((-0.8970045369724385 -0.5240544927...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-25 21:00:00.000,0,SBKP,...,METAF SBKP 252300Z 08008KT CAVOK 21/12 Q1019=\n,SBKP,2023-05-25 22:00:00.000,METAR SBKP 252200Z 09004KT CAVOK 23/13 Q1020=,SBKP,2023-05-25 23:00:00.000,0,SBKP,NaN,NaN


In [1296]:
bimtra_df['solution'] = ((bimtra_df.dt_arr - bimtra_df.dt_dep).dt.total_seconds()).astype(int)
bimtra_df.head()

,flightid,origem,destino,dt_dep,dt_arr,solution
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547


In [1297]:
display(test_data.head())

display(bimtra_df.head())
display(esperas_df.head())
# display(f'metaf_df: {metaf_df.columns}')
# display(f'metar_df: {metar_df.columns}')
display(tc_prev_df.head())
display(tc_real_df.head())

,flightid,origem,destino,dt_dep,hora_ref,snapshot_radar,path,hora_esperas,esperas,aero_esperas,...,metaf,aero_metaf,hora_metar,metar,aero_metar,hora_tcp,troca,aero_tcp,hora_tcr,aero_tcr
0,7945735584a3297121c4f5ae0de8ecd1,SBKP,SBSV,2023-05-30 11:36:03.000,2023-05-30 11:00:00.000,MULTIPOINT ((-0.8807200448127578 -0.5148453155...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-30 10:00:00.000,0,SBSV,...,NaN,NaN,2023-05-30 11:00:00.000,METAR SBSV 301100Z 12009KT 9999 FEW023 28/21 Q...,SBSV,2023-05-30 12:00:00.000,0,SBSV,NaN,NaN
1,85cb958e8b9d583575cd9cb93e389226,SBFL,SBRJ,2023-05-19 08:26:06.000,2023-05-19 08:00:00.000,MULTIPOINT ((-0.8826266039383116 -0.4926305156...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-19 07:00:00.000,0,SBRJ,...,METAF SBRJ 190900Z 33004KT 9999 BKN033 ...,SBRJ,2023-05-19 08:00:00.000,METAR SBRJ 190800Z 02004KT 9999 FEW010 SCT040 ...,SBRJ,2023-05-19 09:00:00.000,0,SBRJ,NaN,NaN
2,2d4e8cd3947525a12ff16af520beefb9,SBCF,SBGR,2023-05-22 11:52:02.000,2023-05-22 11:00:00.000,MULTIPOINT ((-0.8972512464147154 -0.5215196550...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-22 10:00:00.000,0,SBGR,...,METAF SBGR 221200Z 04003KT CAVOK 19/09 Q1021=\n,SBGR,2023-05-22 11:00:00.000,METAR SBGR 221100Z 07006KT CAVOK 13/12 Q1022=,SBGR,2023-05-22 12:00:00.000,0,SBGR,NaN,NaN
3,24cf317a1494a016a018d9f5cd183639,SBFL,SBSP,2023-05-29 16:07:07.000,2023-05-29 16:00:00.000,MULTIPOINT ((-0.892972783728755 -0.51704889771...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-29 15:00:00.000,2,SBSP,...,METAF SBSP 291700Z 15010KT 9999 OVC033 ...,SBSP,2023-05-29 16:00:00.000,METAR SBSP 291600Z 12010KT 090V160 9999 SCT015...,SBSP,2023-05-29 17:00:00.000,0,SBSP,NaN,NaN
4,370babaf32dc82a6a64bf5b71a3434ed,SBCF,SBKP,2023-05-25 22:16:04.000,2023-05-25 22:00:00.000,MULTIPOINT ((-0.8970045369724385 -0.5240544927...,http://satelite.cptec.inpe.br/repositoriogoes/...,2023-05-25 21:00:00.000,0,SBKP,...,METAF SBKP 252300Z 08008KT CAVOK 21/12 Q1019=\n,SBKP,2023-05-25 22:00:00.000,METAR SBKP 252200Z 09004KT CAVOK 23/13 Q1020=,SBKP,2023-05-25 23:00:00.000,0,SBKP,NaN,NaN


,flightid,origem,destino,dt_dep,dt_arr,solution
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547


,esperas,hora,aero
0,0,2022-06-01 00:00:00,SBBR
1,0,2022-06-01 01:00:00,SBBR
2,0,2022-06-01 02:00:00,SBBR
3,0,2022-06-01 03:00:00,SBBR
4,0,2022-06-01 04:00:00,SBBR


,hora,troca,aero
0,2022-06-01 00:00:00,0,BR
1,2022-06-01 01:00:00,0,BR
2,2022-06-01 02:00:00,0,BR
3,2022-06-01 03:00:00,0,BR
4,2022-06-01 04:00:00,0,BR


,hora,nova_cabeceira,antiga_cabeceira,aero
0,2022-06-01 14:14:03,32,03,FL
1,2022-06-01 18:51:10,18,12,RF
2,2022-06-01 19:12:25,16,13,CF
3,2022-06-01 19:12:38,15,26,GL
4,2022-06-01 19:12:57,15,26,GL


In [1298]:
bimtra_df['hora_ref'] = bimtra_df.dt_dep.apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
bimtra_df.head()

,flightid,origem,destino,dt_dep,dt_arr,solution,hora_ref
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,2022-06-01 00:00:00
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,2022-06-01 00:00:00
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,2022-06-01 00:00:00
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,2022-06-01 00:00:00
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,2022-06-01 01:00:00


In [1299]:
display(f'idsc_test_df: {test_data.columns}')

display(f'bimtra_df: {bimtra_df.columns}')
display(f'esperas_df: {esperas_df.columns}')
# display(f'metaf_df: {metaf_df.columns}')
# display(f'metar_df: {metar_df.columns}')
display(f'tc_prev_df: {tc_prev_df.columns}')
display(f'tc_real_df: {tc_real_df.columns}')

"idsc_test_df: Index(['flightid', 'origem', 'destino', 'dt_dep', 'hora_ref', 'snapshot_radar',\n       'path', 'hora_esperas', 'esperas', 'aero_esperas', 'hora_metaf',\n       'metaf', 'aero_metaf', 'hora_metar', 'metar', 'aero_metar', 'hora_tcp',\n       'troca', 'aero_tcp', 'hora_tcr', 'aero_tcr'],\n      dtype='object')"

"bimtra_df: Index(['flightid', 'origem', 'destino', 'dt_dep', 'dt_arr', 'solution',\n       'hora_ref'],\n      dtype='object')"

"esperas_df: Index(['esperas', 'hora', 'aero'], dtype='object')"

"tc_prev_df: Index(['hora', 'troca', 'aero'], dtype='object')"

"tc_real_df: Index(['hora', 'nova_cabeceira', 'antiga_cabeceira', 'aero'], dtype='object')"

In [1300]:
print(f'bimtra_df: {bimtra_df.shape}')
print(f'esperas_df: {esperas_df.shape}')
print(f'metaf_df: {metaf_df.shape}')
print(f'metar_df: {metar_df.shape}')
print(f'tc_prev_df: {tc_prev_df.shape}')
print(f'tc_real_df: {tc_real_df.shape}')

bimtra_df: (300347, 7)
esperas_df: (99936, 3)
metaf_df: (43938, 3)
metar_df: (106744, 3)
tc_prev_df: (99936, 3)
tc_real_df: (33431, 4)


In [1301]:
tc_prev_df['aero_tcp'] = tc_prev_df['aero']
tc_prev_df = tc_prev_df.drop(['aero'], axis=1)
tc_real_df['aero_tcr'] = tc_real_df['aero']
tc_real_df = tc_real_df.drop(['aero'], axis=1)

display(tc_prev_df.head())
display(tc_real_df.head())
display(tc_real_df.shape)

,hora,troca,aero_tcp
0,2022-06-01 00:00:00,0,BR
1,2022-06-01 01:00:00,0,BR
2,2022-06-01 02:00:00,0,BR
3,2022-06-01 03:00:00,0,BR
4,2022-06-01 04:00:00,0,BR


,hora,nova_cabeceira,antiga_cabeceira,aero_tcr
0,2022-06-01 14:14:03,32,03,FL
1,2022-06-01 18:51:10,18,12,RF
2,2022-06-01 19:12:25,16,13,CF
3,2022-06-01 19:12:38,15,26,GL
4,2022-06-01 19:12:57,15,26,GL


(33431, 4)

In [1302]:
tc_real_df.isna().sum()

hora                0
nova_cabeceira      0
antiga_cabeceira    0
aero_tcr            0
dtype: int64

In [1303]:
esperas_df['aero_esperas'] = esperas_df['aero']
esperas_df = esperas_df.drop(['aero'], axis=1)
esperas_df['hora_esperas'] = esperas_df['hora']
esperas_df = esperas_df.drop(['hora'], axis=1)
esperas_df['hora_ref'] = esperas_df['hora_esperas']
esperas_df['destino'] = esperas_df['aero_esperas']

esperas_df.head()

,esperas,aero_esperas,hora_esperas,hora_ref,destino
0,0,SBBR,2022-06-01 00:00:00,2022-06-01 00:00:00,SBBR
1,0,SBBR,2022-06-01 01:00:00,2022-06-01 01:00:00,SBBR
2,0,SBBR,2022-06-01 02:00:00,2022-06-01 02:00:00,SBBR
3,0,SBBR,2022-06-01 03:00:00,2022-06-01 03:00:00,SBBR
4,0,SBBR,2022-06-01 04:00:00,2022-06-01 04:00:00,SBBR


In [1304]:
print(f'bimtra_df: {bimtra_df.shape}')
print(f'esperas_df: {esperas_df.shape}')
print(f'metaf_df: {metaf_df.shape}')
print(f'metar_df: {metar_df.shape}')
print(f'tc_prev_df: {tc_prev_df.shape}')
print(f'tc_real_df: {tc_real_df.shape}')

bimtra_df: (300347, 7)
esperas_df: (99936, 5)
metaf_df: (43938, 3)
metar_df: (106744, 3)
tc_prev_df: (99936, 3)
tc_real_df: (33431, 4)


In [1305]:
tc_real_df['hora_ref'] = tc_real_df.hora.apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
tc_prev_df['hora_ref'] = tc_prev_df.hora

tc_real_df['hora_tcr'] = tc_real_df.hora
tc_prev_df['hora_tcp'] = tc_prev_df.hora
tc_real_df = tc_real_df.drop(['hora'], axis=1)
tc_prev_df = tc_prev_df.drop(['hora'], axis=1)

tc_real_df['destino'] = tc_real_df.aero_tcr
tc_prev_df['destino'] = tc_prev_df.aero_tcp


display(tc_real_df.head())
display(tc_prev_df.head())

,nova_cabeceira,antiga_cabeceira,aero_tcr,hora_ref,hora_tcr,destino
0,32,03,FL,2022-06-01 14:00:00,2022-06-01 14:14:03,FL
1,18,12,RF,2022-06-01 18:00:00,2022-06-01 18:51:10,RF
2,16,13,CF,2022-06-01 19:00:00,2022-06-01 19:12:25,CF
3,15,26,GL,2022-06-01 19:00:00,2022-06-01 19:12:38,GL
4,15,26,GL,2022-06-01 19:00:00,2022-06-01 19:12:57,GL


,troca,aero_tcp,hora_ref,hora_tcp,destino
0,0,BR,2022-06-01 00:00:00,2022-06-01 00:00:00,BR
1,0,BR,2022-06-01 01:00:00,2022-06-01 01:00:00,BR
2,0,BR,2022-06-01 02:00:00,2022-06-01 02:00:00,BR
3,0,BR,2022-06-01 03:00:00,2022-06-01 03:00:00,BR
4,0,BR,2022-06-01 04:00:00,2022-06-01 04:00:00,BR


In [1306]:
tc_prev_df['destino'] = 'SB' + tc_prev_df['destino'].astype(str)
tc_prev_df['destino'].unique()

tc_real_df['destino'] = 'SB' + tc_real_df['destino'].astype(str)
tc_real_df['destino'].unique()

tc_prev_df['aero_tcp'] = 'SB' + tc_prev_df['aero_tcp'].astype(str)
tc_prev_df['aero_tcp'].unique()

tc_real_df['aero_tcr'] = 'SB' + tc_real_df['aero_tcr'].astype(str)
tc_real_df['aero_tcr'].unique()

array(['SBFL', 'SBRF', 'SBCF', 'SBGL', 'SBSV', 'SBKP', 'SBGR', 'SBPA',
       'SBRJ', 'SBSP', 'SBBR', 'SBCT'], dtype=object)

In [1307]:
train_data = pd.merge(bimtra_df, esperas_df, how='left', on=['hora_ref', 'destino'])
train_data = pd.merge(train_data, tc_prev_df, how='left', on=['hora_ref', 'destino'])
train_data = pd.merge(train_data, tc_real_df, how='left', on=['hora_ref', 'destino'])
# train_data['hora_ref'] = train_data['hora_ref'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [1308]:
display(train_data.sample(10))
display(train_data.shape)


,flightid,origem,destino,dt_dep,dt_arr,solution,hora_ref,esperas,aero_esperas,hora_esperas,troca,aero_tcp,hora_tcp,nova_cabeceira,antiga_cabeceira,aero_tcr,hora_tcr
337834,8d785bc240d3a5406b98695798d8b800,SBRJ,SBSP,2023-05-11 15:14:53,2023-05-11 16:11:02,3369,2023-05-11 15:00:00,0,SBSP,2023-05-11 15:00:00,1,SBSP,2023-05-11 15:00:00,NaN,NaN,NaN,NaT
208995,f1034887c12c394f74e923bf9cefcdc7,SBFL,SBKP,2023-01-13 08:26:33,2023-01-13 09:25:40,3547,2023-01-13 08:00:00,0,SBKP,2023-01-13 08:00:00,1,SBKP,2023-01-13 08:00:00,NaN,NaN,NaN,NaT
174112,fc888302f2e0e7235d661e4e3c8ee527,SBBR,SBSP,2022-12-08 19:04:10,2022-12-08 20:28:45,5075,2022-12-08 19:00:00,1,SBSP,2022-12-08 19:00:00,1,SBSP,2022-12-08 19:00:00,NaN,NaN,NaN,NaT
112182,55df3515775e5ba556cc3e19a3e3b165,SBPA,SBGR,2022-10-04 19:03:41,2022-10-04 20:28:45,5104,2022-10-04 19:00:00,0,SBGR,2022-10-04 19:00:00,0,SBGR,2022-10-04 19:00:00,NaN,NaN,NaN,NaT
282174,94c63ff63b0354fccb35ed9a9c630dd9,SBRJ,SBBR,2023-03-29 16:14:56,2023-03-29 17:40:43,5147,2023-03-29 16:00:00,0,SBBR,2023-03-29 16:00:00,1,SBBR,2023-03-29 16:00:00,NaN,NaN,NaN,NaT
122851,ed74197b059f5c53f587eec42ee73221,SBRJ,SBSP,2022-10-16 15:06:22,2022-10-16 15:47:54,2492,2022-10-16 15:00:00,0,SBSP,2022-10-16 15:00:00,0,SBSP,2022-10-16 15:00:00,35L,12,SBSP,2022-10-16 15:47:54
233824,a540ce4b7ec7c76814c299c8c651527d,SBGR,SBCT,2023-02-07 11:02:02,2023-02-07 11:46:50,2688,2023-02-07 11:00:00,0,SBCT,2023-02-07 11:00:00,0,SBCT,2023-02-07 11:00:00,NaN,NaN,NaN,NaT
292650,b78531cc23c5cdd2b69ee6afc11cdb03,SBSP,SBRJ,2023-04-05 13:22:34,2023-04-05 14:05:52,2598,2023-04-05 13:00:00,0,SBRJ,2023-04-05 13:00:00,0,SBRJ,2023-04-05 13:00:00,02R,20L,SBRJ,2023-04-05 13:38:07
205971,9b9fb50437db702ee033e6733bf8d64c,SBCF,SBRJ,2023-01-10 12:40:42,2023-01-10 13:34:15,3213,2023-01-10 12:00:00,0,SBRJ,2023-01-10 12:00:00,0,SBRJ,2023-01-10 12:00:00,NaN,NaN,NaN,NaT
172863,8d0c4c3085f5bb86764cc9279e8c5534,SBRJ,SBSP,2022-12-07 12:40:13,2022-12-07 13:51:22,4269,2022-12-07 12:00:00,10,SBSP,2022-12-07 12:00:00,1,SBSP,2022-12-07 12:00:00,17R,35L,SBSP,2022-12-07 12:55:28


(340098, 17)

In [1309]:
train_data.isna().sum() 

flightid                 0
origem                   0
destino                  0
dt_dep                   0
dt_arr                   0
solution                 0
hora_ref                 0
esperas                  0
aero_esperas             0
hora_esperas             0
troca                    0
aero_tcp                 0
hora_tcp                 0
nova_cabeceira      230141
antiga_cabeceira    230141
aero_tcr            230141
hora_tcr            230141
dtype: int64

In [1310]:
train_data.shape

(340098, 17)

In [1311]:
train_data = train_data.drop(columns=['nova_cabeceira', 'antiga_cabeceira', 'dt_arr', 'hora_ref', 'hora_esperas', 'dt_dep', 'hora_tcp', 'hora_tcr'], axis=1)

train_data = train_data[['flightid', 'origem', 'destino', 'esperas', 'aero_esperas', 'troca', 'aero_tcp', 'aero_tcr', 'solution']]

train_data.head()

,flightid,origem,destino,esperas,aero_esperas,troca,aero_tcp,aero_tcr,solution
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,0,SBRJ,0,SBRJ,NaN,2463
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,0,SBRJ,0,SBRJ,NaN,2152
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,0,SBRJ,0,SBRJ,NaN,4613
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,0,SBCF,0,SBCF,NaN,2581
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,0,SBFL,0,SBFL,NaN,5547


In [1312]:
test_data = test_data.drop(columns=['snapshot_radar', 'path', 'hora_metaf', 'metaf', 'aero_metaf', 'hora_metar', 'metar', 'aero_metar', 'hora_ref', 'hora_esperas', 'dt_dep', 'hora_tcp', 'hora_tcr'], axis=1)
test_data = test_data[['flightid', 'origem', 'destino', 'esperas', 'aero_esperas', 'troca', 'aero_tcp', 'aero_tcr']]
test_data.head()

,flightid,origem,destino,esperas,aero_esperas,troca,aero_tcp,aero_tcr
0,7945735584a3297121c4f5ae0de8ecd1,SBKP,SBSV,0,SBSV,0,SBSV,NaN
1,85cb958e8b9d583575cd9cb93e389226,SBFL,SBRJ,0,SBRJ,0,SBRJ,NaN
2,2d4e8cd3947525a12ff16af520beefb9,SBCF,SBGR,0,SBGR,0,SBGR,NaN
3,24cf317a1494a016a018d9f5cd183639,SBFL,SBSP,2,SBSP,0,SBSP,NaN
4,370babaf32dc82a6a64bf5b71a3434ed,SBCF,SBKP,0,SBKP,0,SBKP,NaN


In [1313]:
cat_columns = ['origem', 'destino', 'aero_esperas', 'aero_tcp', 'aero_tcr']
train_data, test_data = pd.get_dummies(train_data, columns=cat_columns, dtype=int), pd.get_dummies(test_data, columns=cat_columns, dtype=int)

for column in train_data.columns:
    if column == 'solution':
        continue
    if column not in test_data.columns:
        test_data[column] = 0

In [1314]:
train_data.head()

,flightid,esperas,troca,solution,origem_SBBR,origem_SBCF,origem_SBCT,origem_SBFL,origem_SBGL,origem_SBGR,...,aero_tcr_SBCT,aero_tcr_SBFL,aero_tcr_SBGL,aero_tcr_SBGR,aero_tcr_SBKP,aero_tcr_SBPA,aero_tcr_SBRF,aero_tcr_SBRJ,aero_tcr_SBSP,aero_tcr_SBSV
0,fcb2bf90345705318213ae1307c0f901,0,0,2463,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,c7c5c10716335b048f86d8c52fcba3f2,0,0,2152,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3162de7203a972f071d5a48e8f0f4828,0,0,4613,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8fcd243e9b9eebbe62ab145ce04b6ab5,0,0,2581,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,504a62621cd231d6ab67e674ce538cd3,0,0,5547,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1315]:
train_data.describe().columns


Index(['esperas', 'troca', 'solution', 'origem_SBBR', 'origem_SBCF',
       'origem_SBCT', 'origem_SBFL', 'origem_SBGL', 'origem_SBGR',
       'origem_SBKP', 'origem_SBPA', 'origem_SBRF', 'origem_SBRJ',
       'origem_SBSP', 'origem_SBSV', 'destino_SBBR', 'destino_SBCF',
       'destino_SBCT', 'destino_SBFL', 'destino_SBGL', 'destino_SBGR',
       'destino_SBKP', 'destino_SBPA', 'destino_SBRF', 'destino_SBRJ',
       'destino_SBSP', 'destino_SBSV', 'aero_esperas_SBBR',
       'aero_esperas_SBCF', 'aero_esperas_SBCT', 'aero_esperas_SBFL',
       'aero_esperas_SBGL', 'aero_esperas_SBGR', 'aero_esperas_SBKP',
       'aero_esperas_SBPA', 'aero_esperas_SBRF', 'aero_esperas_SBRJ',
       'aero_esperas_SBSP', 'aero_esperas_SBSV', 'aero_tcp_SBBR',
       'aero_tcp_SBCF', 'aero_tcp_SBCT', 'aero_tcp_SBFL', 'aero_tcp_SBGL',
       'aero_tcp_SBGR', 'aero_tcp_SBKP', 'aero_tcp_SBPA', 'aero_tcp_SBRF',
       'aero_tcp_SBRJ', 'aero_tcp_SBSP', 'aero_tcp_SBSV', 'aero_tcr_SBBR',
       'aero_tcr_SBCF', '

In [1316]:
features = train_data.columns
features = features.drop(['flightid', 'solution'])
target = ['solution']

X_train, X_test, y_train, y_test = train_test_split(train_data[features], train_data[target], test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Linear Regression MSE: {mean_squared_error(y_test, y_pred)}')

model2 = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
y_pred = np.round_(y_pred)
print(y_pred)
print(f'Random Forest Regressor MSE: {mean_squared_error(y_test, y_pred)}')

Linear Regression MSE: 3134198.6320632994


/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[2912. 7570. 2912. ... 2912. 2912. 2912.]
Random Forest Regressor MSE: 1645289.42622758


In [1317]:
X_train, y_train = train_data[features], train_data[target]
X_test = test_data[features]
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = np.round_(y_pred)
print(y_pred)

output = pd.DataFrame({'ID': test_data.flightid, 'solution': y_pred})

output.to_csv('data/submission.csv', index=False)

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[ 7042.  3587.  3660.  3587.  3660.  7589.  2912.  2912.  4997.  7632.
  7046.  5087.  3270.  2912.  2912.  8148.  5087.  2912.  4789.  2912.
  4997.  4997.  3660.  7589.  2912.  2912.  4789.  5087.  5087.  4847.
  2912. 10178.  7589.  2912.  2912.  4997.  2912.  2912.  2912.  3660.
  4847.  2912.  2912.  7632.  4997.  2912.  5270.  7589.  4974.  4997.
  2912.  3587.  7741.  4789.  4997.  2912.  3270.  7046.  3660.  6761.
 10302.  4997.  2912.  6761.  4974.  5414.  3579.  4997.  3660.  2912.
  2912.  9891.  3660.  3728.  4997.  4487.  2912.  2912.  2912.  4847.
  9707.  2912.  7589.  2912.  5087.  2912.  6931.  3587.  2912.  2912.
  2912.  3587. 10301.  2912.  2912.  4997.  3660.  2912.  4847.  2912.]


In [1318]:
#metar_df.metar = metar_df.metar.apply(lambda x: x.replace('=', ' '))
#metar_sample = str(metar_df['metar'].sample(1))


# metar_sample = 'METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018'

# print(metar_sample)
# report = Metar.Metar(metar_sample)
# print(report, '\n')

# print(f'Station_Id: {report.station_id}')
# print(f'Type: {report.type}')
# print(f'Time: {report.time}')
# print(f'Day: {report._day}')
# print(f'Temperature: {report.temp}')
# print(f'Dew Point: {report.dewpt}')
# print(f'Wind Direction: {report.wind_dir}')
# print(f'Visibility: {report.vis}')